In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
from PIL import Image
from sklearn.feature_extraction import image as skImage
from glob import glob
import numpy as np
import keras
import os

# -- DATA GETTER -- #
class DataGetter():
    """CLASS::DataGetter:"""
    def __init__(self, data_dir, img_size, classes, phase='train'):
        self.phase = phase
        self.data_dir = data_dir
        self.img_size = img_size
        self.classes = classes
        self.data_paths = []
        self.data = []
        self.label = []
    
    def load_data(self):
        for category in self.classes:
            self.data_paths.append(glob(self.data_dir+os.sep+self.phase+os.sep+category+os.sep+'*.jpg'))
        for k, category in enumerate(self.data_paths):
            for img_path in category:
                img = load_img(img_path)
                img = img.resize((self.img_size,self.img_size))
                self.data.append(np.expand_dims(img_to_array(img),axis=0))
                self.label.append(k)
    
    def __getitem__(self, key):
        if not isinstance(key, int):
            raise ValueError('Key has to be of type: int')
        return (self.data[key],self.label[key])
    
    def __len__(self):
        return len(self.data)

In [2]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Reshape, Activation
from keras import backend as K

# -- MODEL -- #
class Model_MLP():
    """CLASS::Model_MLP"""
    def __init__(self, config_dict, trained=False):
        model = self.get_model_structure(config_dict)
        if trained:
            model.load_weights(config_dict['weights_path'])
        layer_outputs = [layer.output for layer in model.layers]
        self.model = Model(inputs=model.inputs, outputs=layer_outputs)

    def get_model_structure(self,config_dict):
        model = Sequential()
        size = config_dict['img_size']
        model.add(Reshape((size*size*3,),input_shape=(size, size, 3)))
        for layer in config_dict['layers']:
            model.add(Dense(units=layer['units'], activation=layer['activation']))
        model.add(Dense(units=8, activation='softmax'))
        return model

    def predict(self,tensor):
        return self.model.predict(tensor)


In [ ]:
pwd

'/content'

In [ ]:
import zipfile
import os
with zipfile.ZipFile("MIT_split-20210124T132955Z-001.zip", 'r') as zip_ref:
    zip_ref.extractall(os.getcwd())

In [ ]:
path1 = os.getcwd() + "/" + "MIT_split"
print(path1)

/content/MIT_split


In [3]:
import zipfile
import os
with zipfile.ZipFile("MIT_split-20210124T132955Z-001.zip", 'r') as zip_ref:
    zip_ref.extractall(os.getcwd())

path1 = os.getcwd() + "/" + "MIT_split"
print(path1)

/content/MIT_split


In [4]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Reshape
from keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

# -- DEFINE CONSTANTS -- #
config_dict = {
    'layers':[{'units':2048,'activation':'relu'},{'units':1024,'activation':'relu'}],
    'img_size':64,
    'weights_path': os.getcwd() + '/model6.h5'
    }
classes = ['coast','forest','highway','inside_city','mountain','Opencountry','street','tallbuilding']
data_dir = path1
 
if __name__ == "__main__":

    print('Load Model')
    # -- LOAD Model -- #
    model = Model_MLP(config_dict)
    print(config_dict['weights_path'])
    print('Loaded')

    # -- LOAD DATA -- #
    print('Loading train data...')
    train_getter = DataGetter(data_dir, config_dict['img_size'], classes)
    train_getter.load_data()
    print('Loaded.')

    # -- PREDICT DATA -- #
    print('Predicting train data...')
    outputs = []
    labels = []
    for k,(tensor,label) in enumerate(train_getter):
        outputs.append(model.predict(tensor))
        labels.append(label)
        print('Image [{0}] Predicted.'.format(k))
    print('Done')

    with open('./task2_train_feat.pkl','wb') as file:
        pickle.dump(outputs,file)
    with open('./task2_train_label.pkl','wb') as file:
        pickle.dump(labels,file)
    
    # -- LOAD DATA -- #
    print('Loading test data...')
    test_getter = DataGetter(data_dir, config_dict['img_size'], classes, phase='test')
    test_getter.load_data()
    print('Loaded.')

    # -- PREDICT DATA -- #
    print('Predicting test data...')
    outputs = []
    labels = []
    for k,(tensor,label) in enumerate(test_getter):
        outputs.append(model.predict(tensor))
        labels.append(label)
        print('Image [{0}] Predicted.'.format(k))
    print('Done')

    with open('./task2_test_feat.pkl','wb') as file:
        pickle.dump(outputs,file)
    with open('./task2_test_label.pkl','wb') as file:
        pickle.dump(labels,file)

Load Model
/content/model6.h5
Loaded
Loading train data...
Loaded.
Predicting train data...
Image [0] Predicted.
Image [1] Predicted.
Image [2] Predicted.
Image [3] Predicted.
Image [4] Predicted.
Image [5] Predicted.
Image [6] Predicted.
Image [7] Predicted.
Image [8] Predicted.
Image [9] Predicted.
Image [10] Predicted.
Image [11] Predicted.
Image [12] Predicted.
Image [13] Predicted.
Image [14] Predicted.
Image [15] Predicted.
Image [16] Predicted.
Image [17] Predicted.
Image [18] Predicted.
Image [19] Predicted.
Image [20] Predicted.
Image [21] Predicted.
Image [22] Predicted.
Image [23] Predicted.
Image [24] Predicted.
Image [25] Predicted.
Image [26] Predicted.
Image [27] Predicted.
Image [28] Predicted.
Image [29] Predicted.
Image [30] Predicted.
Image [31] Predicted.
Image [32] Predicted.
Image [33] Predicted.
Image [34] Predicted.
Image [35] Predicted.
Image [36] Predicted.
Image [37] Predicted.
Image [38] Predicted.
Image [39] Predicted.
Image [40] Predicted.
Image [41] Predi

In [5]:
import numpy as np

def histogram_intersection_kernel(a, b):
    K = np.empty(shape=(a.shape[0], b.shape[0]), dtype=np.float32)
    for i in range(a.shape[0]):
        K[i] = np.sum(np.minimum(a[i], b), axis=1)
    return K

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [6]:
class Model_MLPatches():
    """CLASS::Model_MLPatches"""
    def __init__(self,config_dict, phase='train', trained=False):
        self.model = self.get_model_structure(config_dict, phase)
        if trained:
            self.model.load_weights(config_dict['weights_path'])

    def get_model_with_layer_name(self,layer_name):
        return Model(inputs=self.model.inputs,outputs=self.model.get_layer(layer_name).output)
        
    def get_model_structure(self,config_dict, phase):
        model = Sequential()
        size = config_dict['img_size']
        model.add(Reshape((size*size*3,),input_shape=(size, size, 3)))
        for k,layer in enumerate(config_dict['layers']):
            model.add(Dense(units=layer['units'], activation=layer['activation'],name='dense{0}'.format(k)))
        if phase is 'train':
            model.add(Dense(units=8, activation='softmax'))
        else:
            model.add(Dense(units=8, activation='linear'))
        return model

    def predict(self,tensor):
        return self.model.predict(tensor)

In [7]:
import numpy as np

from sklearn.cluster import MiniBatchKMeans


class VisualWords():

    def __init__(self, n_clusters):
        self.n_clusters = n_clusters

    def fit(self, descriptors):
        self.codebook = MiniBatchKMeans(n_clusters=self.n_clusters,
            verbose=False,
            batch_size=self.n_clusters*20,
            compute_labels=False,
            reassignment_ratio=10**-4,
            random_state=42)
        descriptors = np.vstack(descriptors)
        self.codebook.fit(descriptors)

    def get_visual_words(self, descriptors):
        visual_words = np.empty((len(descriptors), self.n_clusters), dtype=np.float32)
        for i, descriptor in enumerate(descriptors):
            words = self.codebook.predict(descriptor)
            visual_words[i,:] = np.bincount(words, minlength=self.n_clusters)
        return visual_words

class VisualWordsPyramid():

    def __init__(self, n_clusters):
        self.n_clusters = n_clusters

    def fit(self, descriptors):
        self.codebook = MiniBatchKMeans(n_clusters=self.n_clusters,
            verbose=False,
            batch_size=self.n_clusters*20,
            compute_labels=False,
            reassignment_ratio=10**-4,
            random_state=42)
        full_descriptors = []
        for des in descriptors:
            to_append = []
            for subdes in des:
                to_append.extend(subdes)
            full_descriptors.append(to_append)
        full_descriptors = np.vstack(full_descriptors)
        self.codebook.fit(full_descriptors)

    # def get_visual_words(self, descriptors):
    #     visual_words = np.empty((len(descriptors), self.n_clusters*len(descriptors[0])), dtype=np.float32)
    #     for i, descriptor in enumerate(descriptors):
    #         for j, subdescriptor in enumerate(descriptor):
    #             words = self.codebook.predict(subdescriptor)
    #             visual_words[i,j*self.n_clusters:(j+1)*self.n_clusters] = np.bincount(words, minlength=self.n_clusters)
    #     return visual_words

    def get_visual_words(self, descriptors):
        visual_words = np.empty((len(descriptors), self.n_clusters*(len(descriptors[0])+1)), dtype=np.float32)
        for i, descriptor in enumerate(descriptors):
            full_descriptor = []
            for subdes in descriptor:
                full_descriptor.extend(subdes)
            words = self.codebook.predict(full_descriptor)
            visual_words[i,:self.n_clusters] = np.bincount(words, minlength=self.n_clusters)

            for j, subdescriptor in enumerate(descriptor):
                words = self.codebook.predict(subdescriptor)
                visual_words[i,(j+1)*self.n_clusters:(j+2)*self.n_clusters] = np.bincount(words, minlength=self.n_clusters)
        return visual_words

# SVM Single Input Feature

In [11]:
import numpy as np
import pickle
import os

#from utils.Kernels import histogram_intersection_kernel
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

LAYER_IDX = 2
BASE_PATH = os.getcwd()

train_data_path = BASE_PATH + "/task2_train_feat.pkl"
test_data_path = BASE_PATH + "/task2_test_feat.pkl"

train_data = pickle.load(open(train_data_path,'rb'))
feature_train = np.array([item[LAYER_IDX][0] for item in train_data])
test_data = pickle.load(open(test_data_path,'rb'))
feature_test = np.array([item[LAYER_IDX][0] for item in test_data])

train_labels_path = BASE_PATH + "/task2_train_label.pkl"
test_labels_path = BASE_PATH + "/task2_test_label.pkl"
train_labels = pickle.load(open(train_labels_path,'rb'))
test_labels = pickle.load(open(test_labels_path,'rb'))

print(feature_train.shape)
print(len(train_labels))
K_FOLDS = 5
PARAM_GRID = {'C': [0.001389, 0.01, 0.1, 1, 10], 'kernel': ['rbf', histogram_intersection_kernel], 'gamma': [1e-3, 1e-4, 'scale']}

cv = GridSearchCV(SVC(), param_grid=PARAM_GRID, cv=K_FOLDS, n_jobs=-1, verbose=5)
cv.fit(feature_train, train_labels)

train_score = cv.score(feature_train, train_labels)
test_score  = cv.score(feature_test, test_labels)

print("Train accuracy score: {}\nTest accuracy score: {}\nBest params: {}\n".format(train_score, test_score, cv.best_params_))
print("All results: {}".format(cv.cv_results_))

(1881, 1024)
1881
Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 15.7min finished


Train accuracy score: 0.9526847421584264
Test accuracy score: 0.620817843866171
Best params: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}

All results: {'mean_fit_time': array([ 5.61514883, 15.15721059,  5.71265965, 15.3429605 ,  5.33400226,
       15.12924261,  5.6547708 , 14.98197117,  5.69425597, 14.97090306,
        5.36087785, 15.05010376,  5.67333007, 14.98763895,  5.73073449,
       14.98924003,  5.10146403, 15.23278399,  5.70405507, 15.18755212,
        5.76694479, 15.2340148 ,  3.82647476, 14.89679809,  5.71975594,
       14.86538424,  5.78769331, 14.82185264,  3.69747849, 13.98925352]), 'std_fit_time': array([0.0300062 , 0.21325873, 0.06786433, 0.26609137, 0.04210897,
       0.22498347, 0.04230156, 0.24823977, 0.04400057, 0.23149026,
       0.02965841, 0.24084655, 0.02361113, 0.23760768, 0.02625505,
       0.21383388, 0.02047262, 0.12768683, 0.04316531, 0.16361451,
       0.03278108, 0.26076713, 0.03981933, 0.28304115, 0.02118541,
       0.25030641, 0.04239224, 0.15476924, 0.

Here we are getting a train accuracy of 95% and a test accuracy of 62% 